<i>DISCLAIMER:<br>
Bellow code is only for informative purpose, i renounce any responsibility regarding this code and bellow trained model causing loss of money during trading on real market.</i>

# Stock price prediction : AAPL

This page does following:

- Libraries and trained model from previous step are loaded

- Defined customized functions as well as loss function

- Data are pulled, indicators are applied and prediction is made

<b>Import libraries and customized loss function and define customized function</b>

Bellow code defines customized function that are processing loaded data from stock market and makes prediction. Dataframe is normalized and reverted back via following formulas

<i>Linear scaling formula: x ′ = ( x − x m i n ) / ( x m a x − x m i n )</i><br>
<i>Reverse linear scaling: x = (x * (maxv-minv)) + minv</i>

In [1]:
#Libraries
import tensorflow as tf
import yfinance as yf
import talib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

#Customized loss function
def sign_penalty(y_true,y_pred):
    penalty = 100.
    loss = tf.where(tf.less(y_true*y_pred,0),
                    penalty * tf.square(y_true-y_pred),
                    tf.square(y_true - y_pred)
                   )
    
    return(tf.reduce_mean(loss,axis=-1))

tf.keras.losses.sign_penalty = sign_penalty

#Customized functions
def norm_df(pred1):
    df_temp = pd.DataFrame()
    try:
        pred1 = pred1.drop('Date',axis=1)
    except:
        pass
    pred_np = pred1.to_numpy()
    maxv = np.max(pred_np)
    minv = np.min(pred_np)
    df_temp = (pred1.iloc[:,:]-minv)/(maxv-minv)
    
    return df_temp, maxv,minv
def revert_df(df,maxv,minv):
    df_temp = pd.DataFrame() 
    df_temp = (df.iloc[:,:]*(maxv-minv))+minv
    
    return df_temp
def revert_prediction(value,maxv,minv):
    
    return (value * (maxv-minv))+minv
def MakePred(series,model):
    pr = series.to_numpy()
    series2=np.array([[pr]])   
    pred = tf.data.Dataset.from_tensor_slices(series2)
    prediction = model.predict(pred)
        
    return prediction  
def Predict(pred):
    df_temp1, maxv, minv = norm_df(pred)
    pr = MakePred(df_temp1,model)
    prediction = revert_prediction(pr,maxv,minv)
    prediction = np.squeeze(prediction)
    #print("Prediction is: ",prediction)
    return prediction


<b>Load trained model</b>

In [2]:
model = tf.keras.models.load_model('aapl_model_checkpoint.h5', custom_objects={'sign_penalty': sign_penalty} )
model

<b>Load data from stock market</b>

Bellow code will load last 2 months of data via yahoo finance. Exponential moving averages are placed and formations are being selected. If condition is passed, we can run prediction, <b>if condition is not met, we should not trade today.</b>

In [4]:
stock = yf.download('aapl', 
                      start='2021-10-27', 
                      end='2021-12-31', 
                      interval="1d",
                      progress=False,
)
#Set Indicators
stock['EMA4'] = talib.EMA(stock['Close'], timeperiod=4)
stock['EMA8'] = talib.EMA(stock['Close'], timeperiod=8)
stock['EMA16'] = talib.EMA(stock['Close'], timeperiod=16)
stock = stock.drop(labels=['Adj Close','Volume'],axis=1)


#Get final dataframe
trading_formation = stock.iloc[len(stock)-3:]
trading_formation = trading_formation.reset_index()
    
#print("Data loaded in dataframe trading_formation")

#Check condition
if ((trading_formation.iloc[2,1]>trading_formation.iloc[2-1,1])\
    and (trading_formation.iloc[2,1] > trading_formation.iloc[2,4])\
    and (trading_formation.iloc[2,4] > trading_formation.iloc[2,5])\
    and (trading_formation.iloc[2,5] > trading_formation.iloc[2,6])):     
    print("Trading condition passed, you can make prediction")


else:
    print("condition NOT passed, do NOT trade")

condition NOT passed, do NOT trade


<b>Make Prediction</b>

In [5]:
trading_formation

,Date,Open,High,Low,Close,EMA4,EMA8,EMA16
0,2021-12-28,180.160004,181.330002,178.529999,179.289993,177.892214,176.192975,173.308156
1,2021-12-29,179.330002,180.630005,178.139999,179.380005,178.487331,176.901204,174.022491
2,2021-12-30,179.470001,180.570007,178.089996,178.199997,178.372397,177.189824,174.513962


In [6]:
penalisation = 0.049

pred = Predict(trading_formation)
print("Raw prediction is: ",pred)
ppred = round(pred-penalisation,5)
print("Penalised Prediction is: ",ppred)

Raw prediction is:  181.11569
Penalised Prediction is:  181.06669
